# Bose-Hubbard Model
*****
The Bose-Hubbard Hamiltonian is given by: $$\hat{H}=-J\sum_{\langle i,j\rangle}\left(\hat{b}_{i}^\dagger\hat{b}_{j}+\hat{b}_{i}\hat{b}_{j}^\dagger\right)+\frac{U}{2}\sum_{i} \hat{n}_i\left(\hat{n}_i-\hat{I}\right)+V\sum_{\langle i,j\rangle}\hat{n}_i\hat{n}_j-\mu\sum_{i}\hat{n}_i \$$

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import *

Suppose we have two wells and one total particle in the system. Try computing the Hamiltonian for this system with $V = 0$.

In [5]:
basis1 = basis(2,0)
basis2 = basis(2,1)
def number(N):
    return(create(N)*destroy(N))
number(3)

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 2.]]

In [6]:
def NrNbor(Sites):
    l=[]
    for i in range(Sites):
        for k in range(2):
            l1=[]
            l1.append(i)
            l1.append((i-1)+2*k)
            l.append(l1)
    l.remove(l[2*Sites-1])
    l.remove(l[0])
    return(l)

NrNbor(5)

[[0, 1], [1, 0], [1, 2], [2, 1], [2, 3], [3, 2], [3, 4], [4, 3]]

In [9]:
def ldestroy(N):
    a = basis(N).dag()*create(N)*basis(N)
    return(a)

b = create(2)*destroy(2)+destroy(2)*create(2)
b

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 1.]]

In [8]:
def BHHam(Sites,J=1,U=2,V=0,mu=1):
    -J*

SyntaxError: invalid syntax (<ipython-input-8-1211cae2f09d>, line 2)